# HW 1 - Carlos Alvarado

### Problem 1

In [1]:
import os
import pandas as pd
import requests

def get_and_load_data(data_info):
    '''
    Loads "data_name.csv" from local disk, or downloads it if it's not present
    
    Input: dictionary with information about required data
    Returns: pandas data_frame for "data_name"
    '''
    
    filepath = './data/{}.csv'.format(data_info['source'])
    
    if os.path.exists(filepath):
        data = pd.read_csv(filepath)
    else:
        data = helper_download_data(filepath, data_name)
    
    for old_var, new_var in data_info['rename']:
        data.rename(index=str, columns={old_var: new_var}, inplace=True)
        
    return data
    
def helper_download_data(filepath, data_name):
    offset = 0
    limit = 50000
    data = pd.DataFrame({})
    
    while True:
        
        url = 'https://data.cityofchicago.org/resource/{}.json?$limit={}&$offset={}'.format(
            data_name, limit, offset)
        print('getting data from', url)
    
        response = requests.get(url)
        response_code = response.status_code
        
        if response_code != 200: 
            print('Failed to download data')
            break
            
        json_data = response.content

        if len(json_data) > 4:
            data = pd.concat([data, pd.read_json(json_data)])
        
            offset = offset + limit
        else:
            print(json_data)
            break
            
    #save data to csv for future use
    data.to_csv(filepath)
    
    return data
    

In [2]:
DATA1 = {'name': 'Graffiti Removal',
         'source': 'hec5-y4x5',
         'rename': [('where_is_the_graffiti_located_', 'Sub Type')]    
        }

DATA2 = {'name': 'Vacant and Abandoned Buildings Reported',
         'source': '7nii-7srd',
         'rename': [('is_building_open_or_boarded_', 'Sub Type'), 
                    ('date_service_request_was_received', 'creation_date')]
        }

DATA3 = {'name': 'Pot Holes Reported',
         'source': '7as2-ds3y',
         'rename': [('zip', 'zip_code'), 
                    ('type_of_service_request', 'Sub Type')]
        }
    
DATA4 = {'name': 'Sanitation Code Complaints',
         'source': 'me59-5fac',
         'rename': [('what_is_the_nature_of_this_code_violation_', 'Sub Type')]
        }

SOURCES = [DATA1, DATA2, DATA3, DATA4]

complaints = pd.DataFrame({})

for db_data in SOURCES:
    data = get_and_load_data(db_data)
    print(data.columns)
    data['Complaint Type'] = db_data['name']
    complaints = pd.concat([complaints, data], ignore_index=True)

Index(['Unnamed: 0', 'community_area', 'completion_date', 'creation_date',
       'latitude', 'location', 'longitude', 'police_district',
       'service_request_number', 'ssa', 'status', 'street_address',
       'type_of_service_request', 'ward',
       'what_type_of_surface_is_the_graffiti_on_', 'Sub Type', 'x_coordinate',
       'y_coordinate', 'zip_code'],
      dtype='object')
Index(['Unnamed: 0', 'address_street_direction', 'address_street_name',
       'address_street_number', 'address_street_suffix',
       'any_people_using_property_homeless_childen_gangs_', 'community_area',
       'creation_date', 'if_the_building_is_open_where_is_the_entry_point_',
       'Sub Type', 'is_the_building_currently_vacant_or_occupied_',
       'is_the_building_dangerous_or_hazardous_',
       'is_the_building_vacant_due_to_fire_', 'latitude', 'location',
       'location_of_building_on_the_lot_if_garage_change_type_code_to_bgd_',
       'longitude', 'police_district', 'service_request_number',
 

/usr/local/lib/python3.4/dist-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (3,11) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
complaints.loc[10]

In [19]:
ctcounts = complaints['Complaint Type'].value_counts()
ctcounts

Graffiti Removal                           854622
Pot Holes Reported                         457442
Sanitation Code Complaints                 118761
Vacant and Abandoned Buildings Reported     58716
Name: Complaint Type, dtype: int64

In [36]:
gb = complaints.groupby(['Complaint Type', 'Sub Type']).size().to_frame()
print(gb)

                                                                                         0
Complaint Type                          Sub Type                                          
Graffiti Removal                        Alley                                        35316
                                        Bench                                         2722
                                        Door                                         22303
                                        Dumpster                                     14342
                                        Express Way Job                               1610
                                        Fence                                        16245
                                        Front                                       310521
                                        Garage                                      100033
                                        Garbage Cart                                  6293

In [37]:
gb_community_area = complaints.groupby(['Complaint Type', 'community_area']).size().to_frame()
select = gb_community_area['0'] 
print(gb_community_area)

                                                            0
Complaint Type                          community_area       
Graffiti Removal                        0                 268
                                        1               12334
                                        2               17053
                                        3               11907
                                        4               18178
                                        5               17562
                                        6               23748
                                        7               15078
                                        8               11588
                                        9                 234
                                        10               1523
                                        11               3458
                                        12               1271
                                        13               4860
        

### Problem 2

### Problem 3